In [77]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [92]:
PCA_df = pd.read_csv("FeaturesPCA.csv",index_col=[0])
PCA_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.523476,-0.238780,0.704734,-0.251208,-0.704963,0.083239,-0.016321,-0.338960,-0.101965,-0.320476,...,0.015840,0.138977,-0.090525,0.044719,-0.039023,-0.074408,0.051322,0.048233,-0.095420,-0.117281
1,0.170270,-0.465586,-0.305915,-0.404152,-0.197291,0.284409,-0.234281,-0.308266,0.124785,-0.735082,...,-0.052570,0.086144,0.077468,-0.026543,-0.035970,-0.196330,-0.088867,-0.180740,-0.027363,0.073743
2,0.654504,0.359562,-0.358608,-0.289298,-0.006726,-0.024271,-0.087419,0.293584,-0.030858,-0.270481,...,-0.064232,-0.082718,0.154134,-0.006525,-0.048231,-0.018546,0.037308,0.100942,-0.006716,0.212511
3,-0.045029,0.771255,0.576467,-0.112582,-0.453351,-0.281525,0.753308,-0.256638,-0.458042,0.048076,...,-0.001842,-0.047436,-0.093628,-0.014237,0.117447,0.159251,-0.136388,-0.017576,0.007260,0.066017
4,-0.753907,-0.061010,-0.256266,-0.156085,0.005529,0.046611,-0.245375,-0.039222,-0.447446,0.192799,...,0.234802,0.002370,0.107162,0.069440,0.113330,0.017976,-0.023594,-0.014171,-0.010660,0.057907


In [93]:
df = pd.read_csv("drug_onehot_latest.csv",index_col=[0])
df.head()

,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
inpatient.number,,,,,,,,,,,,,,,,,,
722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0


In [80]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(128, input_dim=50
                           , activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=50)
    else:
        model.fit(X_train, y_train, epochs=50, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

In [81]:
def model():
    global q, PCA_df
    count_class_0, count_class_1 = PCA_df[q].value_counts()[0], PCA_df[q].value_counts()[1]

    # Divide by class
    df_class_0 = PCA_df[PCA_df[q] == 0]
    df_class_1 = PCA_df[PCA_df[q] == 1]
    if count_class_0>count_class_1:
        df_class_1_over = df_class_1.sample(count_class_0, replace=True)
        df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)
    else:
        df_class_0_over = df_class_0.sample(count_class_1,replace=True)
        df_test_over = pd.concat([df_class_0_over, df_class_1], axis=0)

    print('Random over-sampling:')
    print(df_test_over[q].value_counts())
    X = df_test_over.drop(q,axis='columns')
    y = df_test_over[q]

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
    # loss = keras.losses.BinaryCrossentropy()
    # weights = -1
    y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

In [94]:
q = "Atorvastatin calcium tablet"

In [95]:
df1 = df[q]
df1 = df1.to_frame()

In [96]:
df1.head()

,Atorvastatin calcium tablet
inpatient.number,
722128,0
723327,1
723617,1
724385,0
725509,0


In [98]:
PCA_df = pd.concat([PCA_df, df1[q].to_frame()], axis = 1)

In [91]:
PCA_df.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,Atorvastatin calcium tablet,Atorvastatin calcium tablet
0,-0.523476,-0.238780,0.704734,-0.251208,-0.704963,0.083239,-0.016321,-0.338960,-0.101965,-0.320476,...,-0.090525,0.044719,-0.039023,-0.074408,0.051322,0.048233,-0.095420,-0.117281,NaN,NaN
1,0.170270,-0.465586,-0.305915,-0.404152,-0.197291,0.284409,-0.234281,-0.308266,0.124785,-0.735082,...,0.077468,-0.026543,-0.035970,-0.196330,-0.088867,-0.180740,-0.027363,0.073743,NaN,NaN
2,0.654504,0.359562,-0.358608,-0.289298,-0.006726,-0.024271,-0.087419,0.293584,-0.030858,-0.270481,...,0.154134,-0.006525,-0.048231,-0.018546,0.037308,0.100942,-0.006716,0.212511,NaN,NaN
3,-0.045029,0.771255,0.576467,-0.112582,-0.453351,-0.281525,0.753308,-0.256638,-0.458042,0.048076,...,-0.093628,-0.014237,0.117447,0.159251,-0.136388,-0.017576,0.007260,0.066017,NaN,NaN
4,-0.753907,-0.061010,-0.256266,-0.156085,0.005529,0.046611,-0.245375,-0.039222,-0.447446,0.192799,...,0.107162,0.069440,0.113330,0.017976,-0.023594,-0.014171,-0.010660,0.057907,NaN,NaN
